# Jugando con TOM desde Fabric con la ayuda de semantic-link
La librería semantic-link está concebida para conectarse desde el Spark de Fabric a un conjunto de datos Power BI publicado también en Fabric y poder utilizar las tablas y las medidas del modelo en la ciencia de datos. Y también para hacer análisis sobre la estructura del modelo y la calidad de los datos.

Revisando el código de semantic-link he descubierto que para conectarse al conjunto de datos utiliza:
- la API REST de Power BI 
- la biblioteca Microsoft.AnalysisServices.Tabular, la misma que utiliza, por ejemplo, Tabular Editor

He querido probar si utilzando directamente la biblioteca Microsoft.AnalysisServices.Tabular se podría modificar el conjunto de datos y para ello he intentando crear:
- unas medidas 
- un grupo de cálculo 

y Eureka!!! lo he logrado :)

Pero... que se pueda hacer no quiere decir que se deba, porque para lograrlo he utilizado partes de semantic-link que son de uso privado de la librería y no se deberían utilizar fuera de ella. 

Los ejemplos son los mismos que utilicé en la charla [Jugando con TOM desde Python](https://www.dataxbi.com/blog/2021/03/03/jugando-con-tom-desde-python-video/)

### Requisitos previos
Antes de utilizar este notebook debe hacer lo siguiente:
- Publicar un conjunto de datos en la misma área de trabajo de Fabric donde se importe este notebook
- Activar la lectura y escritura en el punto de conexión XMLA de la capacidad Fabric asignada al área de trabajo
- Instalar la librería semantic-link con una de estas opciones:
  - pip install semantic-link en el notebbok
  - agregar la librería en la configuración de Spark del área de trabajo

In [12]:
import sempy.fabric as fabric
# Las siguientes librerias son usadas internamente por sempy y no se deberían utilizar
# pero aquí las utilizo en una demo por que soy curioso :)
# Esta librería implementa la conexión al punto XMLA
from sempy.fabric._client._workspace_client import WorkspaceClient
# Esta es la libreria .Net de Microsoft para modelos tabulares y que se carga en Python usando la librería pythonnet
import Microsoft.AnalysisServices.Tabular as tom

StatementMeta(, 21f8ce47-bb2f-48d4-93ec-0aab854b4570, 16, Finished, Available)

### Conexión al conjunto de datos
El conjunto de datos se llama "Ventas Bicicletas" y está publicado en la misma área de trabajo de este notebook.

In [ ]:
ws = WorkspaceClient()
tom_server = ws.get_connection()
tom_database = tom_server.Databases.GetByName('Ventas Bicicletas')

### Creación de una medida YTD para cada medida que exista en el modelo
Se buscan todas las medidas del modelo y por cada una se crea una nueva medida con el sufijo YTD y que utiliza la función TOTALYTD para calcular el acumulado en el año del valor de la medida original.
Además, a cada medida creada se le agrega una anotación para indicar que se creó con Python y itra anotación con el nombre del autor. 


In [ ]:
for t in tom_database.Model.Tables:
    for m in t.Measures:
        new_measure = tom.Measure()

        new_measure.Name = f'{m.Name} YTD'
        new_measure.Expression = f"TOTALYTD ( [{m.Name}], 'Calendario'[Fecha] )"
        new_measure.DisplayFolder = 'YTD'
        new_measure.Description = f'Calcula {m.Name} desde el inicio del año'
        new_measure.FormatString = m.FormatString

        new_annotation = tom.Annotation()
        new_annotation.Name = 'DevTool'
        new_annotation.Value = 'Python'
        new_measure.Annotations.Add(new_annotation)

        new_annotation = tom.Annotation()
        new_annotation.Name = 'Author'
        new_annotation.Value = 'Nelson'
        new_measure.Annotations.Add(new_annotation)

        m.Table.Measures.Add(new_measure)

        display(f'La medida "{new_measure.Name}" se añadió a la tabla "{new_measure.Table.Name}" y a la carpeta "{new_measure.DisplayFolder}"')

tom_database.Model.SaveChanges()

### Creación de un grupo de cálculo
Se crea el grupo de cálculo "Inteligencia de tiempo" con tres items:
- Periódo actual
- Mismo período mes anterior
- Mismo período mes anterior

In [ ]:
calculation_group = tom.CalculationGroup()
calculation_group.Precedence = 0

# Todos los items tendrán el mismo formato
FORMAT_STRING = '"#.00"'

# Item Período Actual
calculation_item = tom.CalculationItem()
calculation_item.Name = 'Actual'
calculation_item.Expression = 'SELECTEDMEASURE()'
calculation_item.Ordinal = 1
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Item MP Mes Aanterior (Mismo Período Mes Anterior)
calculation_item = tom.CalculationItem()
calculation_item.Name = 'MP Mes Anterior'
calculation_item.Expression = 'CALCULATE(SELECTEDMEASURE(), DATEADD(Calendario[Fecha],-1,month))'
calculation_item.Ordinal = 2
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Item MP Año Anterior (Mismo Período Año Anterior)
calculation_item = tom.CalculationItem()
calculation_item.Name = 'MP Año Anterior'
calculation_item.Expression = 'CALCULATE(SELECTEDMEASURE(), DATEADD(Calendario[Fecha],-1,year))'
calculation_item.Ordinal = 3
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Hay que crear una Tabla para el Grupo Calculado
cg_table = tom.Table()
cg_table.Name = 'Inteligencia Tiempo'
cg_table.CalculationGroup = calculation_group

# La tabla necesita una partición
cg_partition = tom.Partition()
cg_partition.Name = 'Particion Grupo Calculado Inteligencia Tiempo'
# Existe una fuente de datos específica para los Grupos Calculados
cg_partition.Source = tom.CalculationGroupSource()

cg_table.Partitions.Add(cg_partition)

# Columna de la tabla, requerida por el grupo de cálculo
cg_column = tom.DataColumn()
cg_column.Name = 'Períodos'
cg_column.DataType = tom.DataType.String

cg_table.Columns.Add(cg_column)

# Anotaciones
new_annotation = tom.Annotation()
new_annotation.Name = 'DevTool'
new_annotation.Value = 'Python'
cg_table.Annotations.Add(new_annotation)

new_annotation = tom.Annotation()
new_annotation.Name = 'Author'
new_annotation.Value = 'Nelson '
cg_table.Annotations.Add(new_annotation)

# Adicionando la tabla al modelo
tom_database.Model.Tables.Add(cg_table)

# Es necesario deshabilitar las medidas implicitas para poder crear grupos de cálculo
tom_database.Model.DiscourageImplicitMeasures = True

# Guardando cambios en el modelo
tom_database.Model.SaveChanges() 